# Importing library

In [1]:
import pandas as pd
from ast import literal_eval
import subprocess
import pyodbc

# Diviser et préparer les fichiers pour la compatibilité des tables de base de données

In [101]:
df=pd.read_csv('cleaned_github_data.csv')
df['project_date']=pd.to_datetime(df['project_date']).dt.date
df.drop(columns=['Unnamed: 0'],inplace=True)

## Technology

In [3]:
technology=list(df.groupby('technology')['technology'].unique().keys())
df_technology=pd.DataFrame(technology,columns=['technology'])
df_technology.index += 1
df_technology.to_csv('data/technology.csv')

## Repository

In [4]:
df_repo=df[['project_date','name']]
df_repo.index+=1
df_repo.to_csv('data/repository.csv')

## Licence

In [5]:
licence=list(df.groupby('licence')['licence'].unique().keys())
df_licence=pd.DataFrame(licence,columns=['licence'])
df_licence.index+=1
df_licence.to_csv('data/licence.csv')

## Owner

In [6]:
owner=list(df.groupby('owner')['owner'].unique().keys())
df_owner=pd.DataFrame(owner,columns=['owner'])
df_owner.index+=1
df_owner.to_csv('data/owner.csv')

## Topics

In [69]:
from ast import literal_eval

topics=[]
for index,value in df['topics'].items():
    for topic in literal_eval(value):
        if(topic!='No topics'):
         topics.append(topic)

#remove duplicated from list
topics=list(dict.fromkeys(topics))

In [70]:
df_topic=pd.DataFrame(topics,columns=['topic'])
df_topic.index+=1
df_topic.to_csv('data/topics.csv')

## Contributor

In [9]:
df=pd.read_csv('Contributor.csv')
contributor=df.groupby('contributor')['contributor'].unique()
df_contributor=pd.DataFrame(contributor.keys(),columns=['contributor'])
df_contributor.index+=1
df_contributor.to_csv('data/contributor.csv')

## Commit

In [10]:
df_commit=pd.read_csv('contributor.csv')

def con_id(con):
   #get the id of contributor based on index
   return df_contributor.index[df_contributor['contributor']==con][0]

def repo_id(repo):
   #get the id of repository based on index 
   return df_repo.index[df_repo['name']==repo][0]

df_commit['contributor']=df_commit['contributor'].apply(con_id)
df_commit['name']=df_commit['name'].apply(repo_id)


In [19]:
df_commit.rename(columns={'name':'id_repository','contributor':'id_contributor','commit':'Count'},inplace=True)
df.drop_duplicates(inplace=True)
df_commit.to_csv('data/commit.csv',index=False)

## Affected_topic

In [64]:
df_customised_tp=df.drop(df.loc[df['topics']=="['No topics']"].index)
df_customised_tp.reset_index(inplace=True)
df_customised_tp.drop(columns=['Unnamed: 0','index'],inplace=True)

In [83]:
def topic_id(topic):
   #get the id of topic based on index 
   return df_topic.index[df_topic['topic']==topic][0]

affected_topic=[]
#loop on dataframe
for i in range(len(df_customised_tp)):
    #Get the index of repo
    rp_id=repo_id(df_customised_tp.loc[i, "name"])
    #Get the index for each topic in list
    for topic in literal_eval(df_customised_tp.loc[i, "topics"]):
        #['repo_id','topic_id']
        affected_topic.append([rp_id,topic_id(topic)])

In [84]:
df_affected_topic=pd.DataFrame(affected_topic,columns=['repo_id','topic_id'])
df_affected_topic.drop_duplicates(inplace=True)
df_affected_topic.to_csv('data/affected_topic.csv',index=False)

## Repository_details

In [117]:
df_repo_details=df.drop(columns=['name','project_date','topics'],axis=1)
df_repo_details.index+=1

In [121]:
def technology_id(technology):
   #get the id of topic based on index 
   return df_technology.index[df_technology['technology']==technology][0]

def owner_id(owner):
   #get the id of topic based on index 
   return df_owner.index[df_owner['owner']==owner][0]

def licence_id(licence):
   #get the id of topic based on index 
   return df_licence.index[df_licence['licence']==licence][0]


df_repo_details['technology']=df_repo_details['technology'].apply(technology_id)

df_repo_details['owner']=df_repo_details['owner'].apply(owner_id)

df_repo_details['licence']=df_repo_details['licence'].apply(licence_id)

In [123]:
df_repo_details.to_csv('data/repo_details.csv')

# Insertion dans la base de données

In [2]:
cnxn_str="DRIVER={Sql Server};SERVER=ELMEHDI;DATABASE=luisi"
cnxn = pyodbc.connect(cnxn_str, autocommit=True)

## Creation des tableau & Indexing columns

In [3]:
with open('DatabaseCreation.sql', 'r') as f:
    data = f.read()
cursor = cnxn.cursor()
cursor.execute(data)

## function to insert data

In [15]:
def bulk_insert(file_path, db_table_name):
    query= f"bulk insert {db_table_name} from '{file_path}' with (FIRSTROW=2,FORMAT='CSV')"
    cursor = cnxn.cursor()
    cursor.execute(query)

## Repository

In [ ]:
bulk_insert(r'ENTER CSV FILE PATH','repository')

## licence

In [ ]:
bulk_insert(r'ENTER CSV FILE PATH','licence')

## Owner

In [5]:
bulk_insert(r'ENTER CSV FILE PATH','[owner]')

## Technology

In [6]:
bulk_insert(r'ENTER CSV FILE PATH','technology')

## topics

In [7]:
bulk_insert(r'ENTER CSV FILE PATH','technology')

## Contributor

In [8]:
bulk_insert(r'ENTER CSV FILE PATH','contributor')

## repo_details

In [17]:
bulk_insert(r'ENTER CSV FILE PATH','repository_details')

## affected_topic

In [ ]:
bulk_insert(r'ENTER CSV FILE PATH','affected_topic')

## Commit

In [53]:
bulk_insert(r'ENTER CSV FILE PATH','[commit]')

## Close connection

In [59]:
# Close the cursor and the connection
cursor.close()
cnxn.close()

# Evaluation des donnes

## Export large data using BCP command

In [ ]:
# Define the BCP command as a list of arguments
bcp_command = [
    "bcp",
    "github.dbo.repository",
    "out",
    "C:\\Users\\Youcode\\Desktop\\repository.csv",
    "-c",
    "-T",
    "-t",
    "'."
]

path_to_run = "C:\\Program Files (x86)\\Microsoft SQL Server\\160\\Tools\\Binn"

try:
    # Execute the BCP command
    subprocess.run(bcp_command, check=True, cwd=path_to_run)
    print("Data exported successfully.")
except subprocess.CalledProcessError as e:
    print(f"Error occurred: {e}")

## Fetch and convert data to Dataframe

In [74]:
#list of repository that build by C#
query_one="select (select [name] from repository where id=r.id_repository) as 'Repository name' from github.dbo.repository_details as r inner join technology as t on r.id_technology=t.id where t.name='C#'"

#Contributor that have total commit upper then 10
query_two="select con.username from contributor as con inner join [commit] as com on con.id=com.id_contributor where com.[count]<10"

#Project that published in 2023 May
query_three="select * from repository as r where DATEDIFF(MONTH,r.[date],'2023-05-31')<1"

In [ ]:
#Establish a connection to the SQL Server database
connection_string = f'DRIVER=SQL Server;SERVER=ELMEHDI;DATABASE=github'
connection = pyodbc.connect(connection_string)

try:
    #Fetch data from the database into a Pandas DataFrame
    dataframe = pd.read_sql(query_one, connection)
except Exception as e:
    print(f"An error occurred: {e}")
finally:
    #Close the database connection
    connection.close()

In [81]:
dataframe

,Repository name
0,SonySemiconductorSolutions/tof-ar-samples-basic
1,zixing131/ddddocrsharp
2,meaf75/GitNity
3,seixaserick/dotnet-webapi-playground
4,clod81/loader_process_hollow_copy_in_chunk
...,...
9790,MarcusKyung/RecipeBox2.Solution
9791,bbaoqaq/wowsCheaterViewer
9792,executeautomation/CSharpFeatures
9793,prasanthsasikumar/TalkToGPT
